In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dateutil.parser import parse
from time import sleep, time
from random import randint
import re
from IPython.core.display import clear_output
from warnings import warn
import numpy as np
from selenium import webdriver 
import cv2 
import pytesseract
from PIL import Image

In [2]:
def is_captcha_page(soup):
    captcha_text="Please verify you are human by typing the text below and clicking submit."
    page_info=soup.find('font',attrs={'class':"pagebodybold"})
    if page_info!=None and captcha_text in page_info.text:
        return True
    return False

In [3]:
def solve_captcha(company_link):
    driver = webdriver.Firefox('/home/udit/projects/chromiumm')
    driver.get(company_link)
    element = driver.find_element_by_id("imgCaptcha")
    element.screenshot('final.png')
    img = cv2.imread('final.png')
    captcha=pytesseract.image_to_string(img)
    
    captcha=re.sub(r'\W+', '', captcha)
    captcha_input = driver.find_element_by_xpath("//input[@id ='txtCaptcha']") 
    captcha_input.send_keys(captcha) 

    # submit button clicked
    driver.find_element_by_xpath("//input[@id ='btnRegiser']").click()
    html = driver.page_source
    soup = BeautifulSoup(html)
    driver.close()
    return soup

In [4]:
def try_filling(strr,company_info):
    try:
        global x
        eqn= "{0} = {1}".format("x",strr)
        x = np.nan
        exec(eqn,globals())
        return x
    except IndexError:
        return(np.nan)

In [5]:
URL = "https://ibanknet.com/scripts/callreports/fiList.aspx?type=bankmanda"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

In [6]:
companies=soup.find_all('a', attrs = {'class':'pagebody'}) 

In [7]:
all_companies=[]
error_links=[]

for company in companies:
    company_link="https://ibanknet.com/"+company.get('href')
    try:
        c = requests.get(company_link)
        soup = BeautifulSoup(c.content, 'html5lib')

        while(is_captcha_page(soup)):
            soup=solve_captcha(company_link)

        company_info=[]

        for info in soup.find_all('td', attrs = {'class':'instinfo'}):
            company_info.append(info.text)


        company_inf={}
        company_inf["company_name"]=soup.find('font',attrs={'class':"blockuhilite"}).text.strip()
        company_inf["RSSD_ID"]=company_info[0].split()[company_info[0].split().index("RSSD")+2]
        company_inf['RefinedAddress']=company_info[1]
        company_inf['City']=company_info[4].split(",")[0].strip()
        company_inf['State']=company_info[4].split(",")[1].strip()
        company_inf['Pin']=company_info[4].split(",")[2].strip()
        company_inf['webAddress']=company_info[7].strip() if ('.com' in company_info[7].strip()) else np.nan
        company_inf['YearOpened']=company_info[company_info.index('Year Opened : ')+1]
        company_inf['FiduciaryPowers']=company_info[company_info.index('Fiduciary Powers : ')+1]
        company_inf['EntityType']=company_info[company_info.index('Entity Type : ')+1]
        company_inf['PowersExercised']=company_info[company_info.index('Powers Exercised : ')+1]
        company_inf['MemberFederalReserve']=company_info[company_info.index('Member Federal Reserve : ')+1]
        company_inf['InstitutionType']=company_info[company_info.index('Institution Type : ')+1]
        company_inf['FederalRegulator']=company_info[company_info.index('Federal Regulator : ')+1]
        company_inf['S-Corp']=company_info[company_info.index('S-Corp : ')+1]
        company_inf['Specialization']=company_info[company_info.index('Specialization : ')+1]

        all_companies.append(company_inf)
        
    except Exception as e:
        error_links.append([company_link,str(e)])

In [8]:
all_compaies=pd.DataFrame(all_companies)

In [9]:
all_compaies.head()

,company_name,RSSD_ID,RefinedAddress,City,State,Pin,webAddress,YearOpened,FiduciaryPowers,EntityType,PowersExercised,MemberFederalReserve,InstitutionType,FederalRegulator,S-Corp,Specialization
0,Peoples Bank,278153,445 South Lewis Avenue,Tulsa,Oklahoma,74150,https://www.peoplesbanktulsa.com,1907,No,State,No,No,Commercial bank,FDIC,Yes,Commercial Lending Specialization
1,American Heritage Bank,311050,2 South Main Street,Sapulpa,Oklahoma,74066,NaN,1905,Yes,State,Yes,Yes,Commercial bank,FED,Yes,All Other Specialization > 1 Billion
2,The Poplar Grove State Bank,808448,109 North State Street,Poplar Grove,Illinois,61065,poplargrovestatebank.com,1946,No,State,No,No,Commercial bank,FDIC,Yes,Agricultural Specialization
3,Forreston State Bank,927134,200 Main Street,Forreston,Illinois,61030,www.fsbil.com,1887,No,State,No,No,Commercial bank,FDIC,Yes,Agricultural Specialization
4,Stock Growers Bank,1009055,324 Main Avenue,Napoleon,North Dakota,58561,NaN,1900,No,State,No,No,Commercial bank,FDIC,Yes,Agricultural Specialization


In [10]:
error_links

[]